# LLM Society Quick Overview

This tutorial provides a concise walkthrough for new users:

1. Information-only propagation (minimal setup)
2. Intervention-only runs
3. Combining information + intervention
4. Key configuration parameters
5. Plotting and dashboards


## 0. Imports


In [ ]:
from llm_society import network
from llm_society.viz import set_theme

set_theme()



## 1. Information-only propagation (minimal)

Only specify the claim text; defaults handle the rest.


In [ ]:
info_only = network(
    information="5G towers cause illness."
)
info_only.simulate()
info_only.plot(type="final")



## 2. Intervention-only run

Omit `information`, and define the intervention parameters.


In [ ]:
intervention_only = network(
    intervention_round=4,
    intervention_nodes=[0, 1, 2],
    intervention_content="Local officials recommend masking indoors.",
    talk_prob=0.3,
    rng=1,
)
intervention_only.simulate()
intervention_only.plot(type="intervention", attr="political", groups=["Democrat", "Republican"])



## 3. Combined information + intervention


In [ ]:
combo = network(
    information="Misinformation about 5G towers.",
    intervention_round=5,
    intervention_nodes=[0, 1, 2, 3],
    intervention_content="Please rely on verified health sources.",
    talk_prob=0.25,
    rng=5,
)
combo.simulate()
combo.plot(type="group", by="traits", attr="political", metric="credibility")



## 4. Key parameters

- `n`, `rounds`, `degree`: network size and connectivity
- `talk_prob`, `edge_frac`, `depth`, `depth_max`: conversation sampling and depth control
- `mode`, `complex_k`, `seeds`, `seed_score`: propagation modes and initial conditions
- `segments`: persona traits and group proportions
- `metrics`: multi-dimensional scoring (e.g., credibility, emotion)
- `intervention_round`, `intervention_nodes`, `intervention_content`: treatment configuration
- `memory_turns_per_agent`: simple memory for agents' prior dialogue


## 5. Plotting and dashboards


In [ ]:
info_only.plot(type="coverage")
info_only.plot(type="centrality", metric="degree")
info_only.plot(type="group", by="traits", attr="political")

fig = info_only.dashboard(engine="plotly", attr="political", metric="credibility")
fig


Export HTML if needed:


In [ ]:
html = info_only.dashboard(engine="plotly", attr="political", metric="credibility", to_html=True)
with open("quickstart_dashboard.html", "w", encoding="utf-8") as f:
    f.write(html)


You're all set! Modify the parameters above to explore different network sizes, interventions, and visualization options.
    